Условие: Задача состоит в модификации кода градиентного спуска для задачи регрессии (ноутбук лекции - Gradient_descent.ipynb). Необходимо, чтобы ваш градиентный спуск мог решать задачи классификации, а именно линейной классификации и логистической регрессии.
На всякий случай напомню, что код будет разный, поэтому жду от вас две функции градиентного спуска.
Чтобы решить задачи, обратите внимание на пункт “Принцип максимального правдоподобия” статьи http://www.machinelearning.ru/wiki/images/6/68/voron-ML-Lin.pdf . Там находится ответ на вопрос “какая функция потерь”.

In [1]:
# first generate a data
import random
import pandas as pd
import numpy as np

# Don't run this cell

In [2]:
data = random.sample(range(1, 1000), 100)
data = pd.DataFrame(data)

# simple class determ function. odd/even
data['guess'] = data[0].apply(lambda x: x % 2)

# Don't run this cell

In [3]:
# the model crashed while trying to predict this
# was predictable, because it is not linear function
# let's do it another way

data = random.sample(range(1, 1000), 100)
data = pd.DataFrame(data)

def if_more(x):
    if x > 500:
        return 1
    else:
        return 0

# simple class determ function. odd/even
data['guess'] = data[0].apply(if_more)
data

,0,guess
0,323,0
1,969,1
2,195,0
3,124,0
4,743,1
...,...,...
95,545,1
96,727,1
97,252,0
98,155,0


In [4]:
# run only if you ran the one of previous two cells
x = data[[0,'w']]
x = x.to_numpy()
y = data['guess']
x[:5]

KeyError: "['w'] not in index"

# Linear classifier

In [5]:
def gradient_descent_class(alpha, x, y, numIterations):
    """
    Linear gradient classifier
    """
    
    m = x.shape[0] 
    
    # add ones for delta
    x = np.c_[ np.ones(m), x]
    
    n = x.shape[1]
    
    theta = np.ones(n) # model weights
    x_transpose = x.transpose()
    
    print(f"Initializing gradient search. M is {m}. N is {n}")
    
    for iter in range( 0, numIterations ):
        hypothesis = np.dot(x, theta) # матричное произведение
        
        loss = hypothesis - y
        J = np.sum(loss ** 2) / (2 * m)  # функция потерь
        
        #print( "iter %s | J: %.3f" % (iter, J) )
        #print(theta)
        
        gradient = np.dot(x_transpose, loss) / m         
        theta = theta - alpha * gradient  # update
    
    print('Model weights:')
    print(theta)
    return theta

In [6]:
# fuck it, take ready dataset
from sklearn import datasets
iris = datasets.load_iris()

prepare_data = pd.DataFrame(iris.data)
prepare_data['class'] = iris.target

# remove third class
prepare_data = prepare_data[prepare_data['class'] != 2] 

x = prepare_data.drop('class', axis=1)
x = x.to_numpy()
y = prepare_data['class']
y = y.to_numpy()

In [7]:
alpha = 0.01
theta = gradient_descent_class(alpha, x, y, 1000)

Initializing gradient search. M is 100. N is 5
Model weights:
[ 0.74630389 -0.25887133  0.04055461  0.17711531  0.68463828]


In [8]:
def class_prediction(row):
    global theta
    return theta[0] + theta[1]*row[0] + theta[2]*row[1] + theta[3]*row[2] + theta[4]*row[3]

results = pd.DataFrame(x)
results['class'] = y
results['predict'] = results.apply(class_prediction, axis=1)
results

,0,1,2,3,class,predict
0,5.1,3.5,1.4,0.2,0,-0.047110
1,4.9,3.0,1.4,0.2,0,-0.015613
2,4.7,3.2,1.3,0.2,0,0.026561
3,4.6,3.1,1.5,0.2,0,0.083816
4,5.0,3.6,1.4,0.2,0,-0.017167
...,...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1,0.957851
96,5.7,2.9,4.2,1.3,1,1.022260
97,6.2,2.9,4.3,1.3,1,0.910536
98,5.1,2.5,3.0,1.1,1,0.811895


In [9]:
from sklearn import linear_model
model = linear_model.SGDRegressor( alpha = 0.01, max_iter = 1000 )

model.fit( x, y )

SGDRegressor(alpha=0.01, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [10]:
results['control'] = results.apply(lambda x: model.intercept_ + model.coef_[0]*x[0] 
                                   + model.coef_[1]*x[1] + model.coef_[2]*x[2] + model.coef_[3]*x[3], axis=1)

In [11]:
results

,0,1,2,3,class,predict,control
0,5.1,3.5,1.4,0.2,0,-0.047110,[-0.0011209870405792452]
1,4.9,3.0,1.4,0.2,0,-0.015613,[0.05936620157813464]
2,4.7,3.2,1.3,0.2,0,0.026561,[0.00553738959618514]
3,4.6,3.1,1.5,0.2,0,0.083816,[0.0694223462366387]
4,5.0,3.6,1.4,0.2,0,-0.017167,[-0.01490753699224203]
...,...,...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1,0.957851,[0.9119914572399236]
96,5.7,2.9,4.2,1.3,1,1.022260,[0.9353528232160067]
97,6.2,2.9,4.3,1.3,1,0.910536,[0.9676410789657733]
98,5.1,2.5,3.0,1.1,1,0.811895,[0.6418027412195866]


# Logistic regression
http://neerc.ifmo.ru/wiki/index.php?title=%D0%9B%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F

In [70]:
import math


def gdc_log(alpha, x, y, numIterations):
    """
    Logistic gradient classifier
    """
    
    m = x.shape[0] 
    
    # add ones for delta
    x = np.c_[ np.ones(m), x]
    
    n = x.shape[1]
    
    w = np.ones(n) # model weights
    x_transpose = x.transpose()
    
    print(f"Initializing gradient search. M is {m}. N is {n}")
    
    for iter in range( 0, numIterations ):
        # гипотеза
        # 1 / (1 + e ** -x*w.transpose) Скалярное произведение 
        # hypothesis = np.dot(x, theta) # матричное произведение
        
        e_power = -1 * np.dot(x, w.transpose()) 
        
        hypothesis = 1 / (1 + np.exp(e_power))
        
        J = -1 * y * np.log(np.dot(x,w)) - (1-y) * np.log(1 - np.dot(x,w))
        
        # J = np.sum(loss ** 2) / (2 * m)  # функция потерь
        
        print(f"iter {iter}, J:")
        print(J)
        print(w)
        
        gradient = np.dot((np.dot(x,w) - y), x)
                        #gradient = np.dot(x_transpose, loss) / m
        # 
        w = w - alpha * gradient  # update
    
    print('Model weights:')
    print(theta)
    return theta

In [71]:
alpha = 0.01
theta = gdc_log(alpha, x, y, 1000)

Initializing gradient search. M is 100. N is 5
iter 0, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[1. 1. 1. 1. 1.]
iter 1, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[-11.717  -69.7189 -38.1718 -37.8924  -9.9666]
iter 2, J:
[nan nan nan nan nan nan nan nan nan nan n

C:\Code\Anaconda\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in log



[-3.59616775e+92 -2.00033695e+93 -1.10697547e+93 -1.09973978e+93
 -3.09827745e+92]
iter 56, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[1.77642405e+94 9.88120385e+94 5.46820387e+94 5.43246126e+94
 1.53047771e+94]
iter 57, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]


[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[1.25061021e+189 6.95641019e+189 3.84963914e+189 3.82447619e+189
 1.07746292e+189]
iter 113, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[-6.17772641e+190 -3.43630643e+191 -1.90163308e+191 -1.88920316e+191
 -5.32241867e+190]
it

iter 162, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[6.05978310e+273 3.37070149e+274 1.86532767e+274 1.85313506e+274
 5.22080464e+273]
iter 163, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[-2.99339330e+275 -1.66504891e+276 -9.21428911e+275 -9.15406042e+275
 -2.5789

...

iter 779, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan]
iter 780, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan]
iter 781, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan n

iter 836, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan]
iter 837, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan]
iter 838, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan n

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan]
iter 883, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan]
iter 884, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan na

iter 930, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan]
iter 931, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan]
iter 932, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan n

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan]
iter 968, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan]
iter 969, J:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan na

In [14]:
# check results by standart libraries 
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs')
model = clf.fit(x, y)

In [19]:
model.coef_

array([[ 0.44036482, -0.90696813,  2.30849566,  0.96232763]])

In [72]:
model.intercept_

array([-6.61165119])

In [20]:
logr = pd.DataFrame(x)
logr['class'] = y
logr['predict'] = logr.apply(lambda x: theta[0] + theta[1]*x[0] + theta[2]*x[1] + theta[3]*x[2] + theta[4]*x[3], axis=1)
logr['control'] = logr.apply(lambda x: model.intercept_[0] + model.coef_[0][0]*x[0] 
                                   + model.coef_[0][1]*x[1] + model.coef_[0][2]*x[2] + model.coef_[0][3]*x[3], axis=1)
logr

,0,1,2,3,class,predict,control
0,5.1,3.5,1.4,0.2,0,-0.047110,-4.115820
1,4.9,3.0,1.4,0.2,0,-0.015613,-3.750409
2,4.7,3.2,1.3,0.2,0,0.026561,-4.250725
3,4.6,3.1,1.5,0.2,0,0.083816,-3.742365
4,5.0,3.6,1.4,0.2,0,-0.017167,-4.250553
...,...,...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1,0.957851,4.027999
96,5.7,2.9,4.2,1.3,1,1.022260,4.214928
97,6.2,2.9,4.3,1.3,1,0.910536,4.665960
98,5.1,2.5,3.0,1.1,1,0.811895,1.350836
